<a href="https://colab.research.google.com/github/UtkarshAIITB/IPL-Data-Analysis/blob/main/DL_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
ball = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IPL Score_Analysis/CSV/IPL Ball-by-Ball 2008-2020.csv')
match = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IPL Score_Analysis/CSV/IPL Matches 2008-2020.csv')
# print(ball.shape)
# ball.head()  

In [4]:
df = pd.merge(match , ball, on='id', how = 'right')
df['date'] = pd.to_datetime(df['date'])
df = df.replace(['Rising Pune Supergiant' , 'Rising Pune Supergiants', 'Delhi Daredevils'], ['Pune Warriors' , 'Pune Warriors', 'Delhi Capitals'])
df['year'] = pd.DatetimeIndex(df['date']).year
columns = ['neutral_venue', 'result', 'result_margin', 'umpire1', 'umpire2', 'batsman', 'non_striker', 'bowler', 'non_boundary', 'dismissal_kind',
           'player_dismissed', 'fielder', 'extras_type', 'player_of_match', 'venue', 'city']
df_run = df.drop(columns = columns)
df_run['method']  = df_run['method'].fillna('Normal')
df_run = df_run.dropna()

In [5]:
df_sort = df_run.sort_values(by = ['id', 'inning', 'over', 'ball'])
df_sort = df_sort.reset_index()
df_sort = df_sort.drop(columns = 'index')
dummy_td = pd.get_dummies(df_sort.toss_decision)
df_enc = pd.concat([df_sort, dummy_td], axis = 'columns')                     #encoded toss decisions - field or bat
df_enc = df_enc.drop(columns = ['toss_decision', 'bat', 'year'])
# df_enc.head()
dummy_td = pd.get_dummies(df_sort.method)
df_enc = pd.concat([df_enc, dummy_td], axis = 'columns')
df_enc = df_enc.drop(columns = ['method', 'Normal'])                          #encoded method - normal or D/L
# df_enc.head()
dummy_td = pd.get_dummies(df_sort.eliminator)
dummy_td.rename(columns = {'Y': 'eliminator?' }, inplace = True )
df_enc = pd.concat([df_enc, dummy_td], axis = 'columns')
df_enc = df_enc.drop(columns = ['eliminator', 'N'])  
df_enc.head()
df_enc['inning_bat'] = df_enc['id'].astype(str)+(df_enc['batting_team']).astype(str)
list1 = df_enc['inning_bat'].unique()
df_copy = df_enc.copy()
df_copy = df_copy.groupby(by = ['inning_bat']).sum()['total_runs'].reset_index()
data = pd.merge(left=df_enc, right=df_copy, on='inning_bat', how='right')
data['over'] = data['over']+ (data['ball'])/10
data.drop(columns = ['ball'], inplace = True)

In [6]:
df_final = pd.DataFrame()
for bat in list1:
  inning  = data[data['inning_bat']==bat].copy()
  # inning.head()
  cum_total  = inning.loc[:,'total_runs_x'].cumsum(axis = 0)
  inning['cum_total'] = cum_total
  cum_wicket = inning.loc[:,'is_wicket'].cumsum(axis = 0)
  inning['cum_wicket'] = cum_wicket
  # inning.head()
  df_final = df_final.append(inning, ignore_index=True)

In [7]:
team_dict = {'Mumbai Indians':12, 'Chennai Super Kings':11, 'Rajasthan Royals':10, 'Kolkata Knight Riders':9, 'Kings XI Punjab':8,
             'Sunrisers Hyderabad':7, 'Delhi Capitals':6, 'Royal Challengers Bangalore':5, 'Gujarat Lions':4, 'Deccan Chargers':3,
             'Pune Warriors':2, 'Kochi Tuskers Kerala':1
             }

In [8]:
bore = df_final.copy()
bore['team1'] = bore.team1.map(team_dict)
bore['team2'] = bore.team2.map(team_dict)
bore['toss_winner'] = bore.toss_winner.map(team_dict)
bore['winner'] = bore.winner.map(team_dict)
bore['batting_team'] = bore.batting_team.map(team_dict)
bore['bowling_team'] = bore.bowling_team.map(team_dict)
bore.head()

,id,date,team1,team2,toss_winner,winner,inning,over,batsman_runs,extra_runs,total_runs_x,is_wicket,batting_team,bowling_team,field,D/L,eliminator?,inning_bat,total_runs_y,cum_total,cum_wicket
0,335982,2008-04-18,5,9,5,9,1,0.1,0,1,1,0,9,5,1,0,0,335982Kolkata Knight Riders,222,1,0
1,335982,2008-04-18,5,9,5,9,1,0.2,0,0,0,0,9,5,1,0,0,335982Kolkata Knight Riders,222,1,0
2,335982,2008-04-18,5,9,5,9,1,0.3,0,1,1,0,9,5,1,0,0,335982Kolkata Knight Riders,222,2,0
3,335982,2008-04-18,5,9,5,9,1,0.4,0,0,0,0,9,5,1,0,0,335982Kolkata Knight Riders,222,2,0
4,335982,2008-04-18,5,9,5,9,1,0.5,0,0,0,0,9,5,1,0,0,335982Kolkata Knight Riders,222,2,0


In [9]:
bore.drop(columns = ['extra_runs', 'inning_bat', 'batsman_runs', 'total_runs_x', 'is_wicket'], inplace = True)
bore.head()

,id,date,team1,team2,toss_winner,winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,total_runs_y,cum_total,cum_wicket
0,335982,2008-04-18,5,9,5,9,1,0.1,9,5,1,0,0,222,1,0
1,335982,2008-04-18,5,9,5,9,1,0.2,9,5,1,0,0,222,1,0
2,335982,2008-04-18,5,9,5,9,1,0.3,9,5,1,0,0,222,2,0
3,335982,2008-04-18,5,9,5,9,1,0.4,9,5,1,0,0,222,2,0
4,335982,2008-04-18,5,9,5,9,1,0.5,9,5,1,0,0,222,2,0


In [10]:
go = bore.copy()
y2 = go['winner']
go.drop(columns = ['id', 'date', 'winner'], inplace = True)
go.head()

,team1,team2,toss_winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,total_runs_y,cum_total,cum_wicket
0,5,9,5,1,0.1,9,5,1,0,0,222,1,0
1,5,9,5,1,0.2,9,5,1,0,0,222,1,0
2,5,9,5,1,0.3,9,5,1,0,0,222,2,0
3,5,9,5,1,0.4,9,5,1,0,0,222,2,0
4,5,9,5,1,0.5,9,5,1,0,0,222,2,0


In [11]:
y = go.iloc[:,-3].values
y = y.reshape(-1,1)
go.drop(columns = ['total_runs_y'], inplace = True)

In [12]:
print(y.shape)
print(go.shape)
go.head()

(193096, 1)
(193096, 12)


,team1,team2,toss_winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,cum_total,cum_wicket
0,5,9,5,1,0.1,9,5,1,0,0,1,0
1,5,9,5,1,0.2,9,5,1,0,0,1,0
2,5,9,5,1,0.3,9,5,1,0,0,2,0
3,5,9,5,1,0.4,9,5,1,0,0,2,0
4,5,9,5,1,0.5,9,5,1,0,0,2,0


In [16]:
X = go.copy()     

In [17]:
def custom_accuracy2(y_test,y_pred,thresold):
    right = 0
    l = len(y_pred)
    for i in range(0,l):
        if(abs(y_pred[i]-y_test[i]) <= thresold):
            right += 1
    return ((right/l)*100)

In [18]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [20]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
  
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [25]:
model = Sequential()
  
model.add(Dense(43, activation='relu'))
model.add(Dropout(0.5))
  
model.add(Dense(22, activation='relu'))
model.add(Dropout(0.5))
  
model.add(Dense(11, activation='relu'))
model.add(Dropout(0.5))
  
model.add(Dense(1))
  
model.compile(optimizer='adam', loss='mse')

In [29]:
model.fit(x=X_train, y=y_train, epochs=400, 
          validation_data=(X_test,y_test))

Epoch 1/400
5431/5431 [==============================] - 11s 2ms/step - loss: 5459.6743 - val_loss: 1405.5427
Epoch 2/400
5431/5431 [==============================] - 10s 2ms/step - loss: 3545.1462 - val_loss: 941.7726
Epoch 3/400
5431/5431 [==============================] - 10s 2ms/step - loss: 3002.9788 - val_loss: 717.7103
Epoch 4/400
5431/5431 [==============================] - 10s 2ms/step - loss: 2652.0505 - val_loss: 712.5739
Epoch 5/400
5431/5431 [==============================] - 10s 2ms/step - loss: 2480.3750 - val_loss: 673.1611
Epoch 6/400
5431/5431 [==============================] - 10s 2ms/step - loss: 2335.2781 - val_loss: 729.0648
Epoch 7/400
5431/5431 [==============================] - 10s 2ms/step - loss: 2224.5583 - val_loss: 677.3342
Epoch 8/400
5431/5431 [==============================] - 10s 2ms/step - loss: 2105.7627 - val_loss: 649.1936
Epoch 9/400
5431/5431 [==============================] - 10s 2ms/step - loss: 1990.5479 - val_loss: 679.7069
Epoch 10/400
5431/

In [30]:
predictions = model.predict(X_test)
sample = pd.DataFrame(predictions,columns=['Predict'])
sample['Actual']=y_test
sample.head(10)

,Predict,Actual
0,147.073669,166
1,141.973022,121
2,154.086243,164
3,137.732468,137
4,132.450165,115
5,164.958496,162
6,135.361252,71
7,158.057129,161
8,147.073669,137
9,151.294022,136


In [31]:
custom_accuracy2(sample['Actual'] , sample['Predict'],10)

41.693423096841016

In [34]:
go.shape

(193096, 12)

In [39]:
byover = bore[bore.over == 7.1].reset_index().drop(columns = 'index')
byover.head()

,id,date,team1,team2,toss_winner,winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,total_runs_y,cum_total,cum_wicket
0,335982,2008-04-18,5,9,5,9,1,7.1,9,5,1,0,0,222,68,1
1,335982,2008-04-18,5,9,5,9,2,7.1,5,9,1,0,0,82,34,4
2,335983,2008-04-19,8,11,11,11,1,7.1,11,8,0,0,0,240,62,3
3,335983,2008-04-19,8,11,11,11,2,7.1,8,11,0,0,0,207,73,1
4,335984,2008-04-19,6,10,10,6,1,7.1,10,6,0,0,0,129,45,3


In [40]:
byover.drop(columns = ['id', 'date', 'winner'], inplace = True)

In [41]:
y = byover.iloc[:,-3].values
y = y.reshape(-1,1)

In [42]:
print(y)

[[222]
 [ 82]
 [240]
 ...
 [172]
 [156]
 [157]]


In [43]:
byover.drop(columns = ['total_runs_y'], inplace = True)

In [44]:
X = byover.copy()

In [45]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [47]:
from sklearn.ensemble import RandomForestRegressor
rf = make_pipeline(StandardScaler(), RandomForestRegressor())
rf.fit(X_train, y_train.ravel())

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestregressor', RandomForestRegressor())])

In [48]:
y_predict = rf.predict(X_test)
print("Custom accuracy: {:.3f} %".format(custom_accuracy2(y_test,y_predict,10)))

Custom accuracy: 30.864 %


In [51]:
# print(y_predict)

In [50]:
y_lol = rf.predict(X_train)
print("Custom accuracy: {:.3f} %".format(custom_accuracy2(y_train,y_lol,10)))

Custom accuracy: 73.883 %


In [52]:
from sklearn.svm import SVR

In [53]:
svr = make_pipeline(StandardScaler(), SVR())

In [54]:
svr.fit(X_train, y_train.ravel())

Pipeline(steps=[('standardscaler', StandardScaler()), ('svr', SVR())])

In [55]:
y_predict = rf.predict(X_test)
print("Custom accuracy: {:.3f} %".format(custom_accuracy2(y_test,y_predict,10)))

Custom accuracy: 30.864 %


In [56]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
  
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [57]:
model = Sequential()
  
model.add(Dense(43, activation='relu'))
model.add(Dropout(0.5))
  
model.add(Dense(22, activation='relu'))
model.add(Dropout(0.5))
  
model.add(Dense(11, activation='relu'))
model.add(Dropout(0.5))
  
model.add(Dense(1))
  
model.compile(optimizer='adam', loss='mse')

In [58]:
model.fit(x=X_train, y=y_train, epochs=400, 
          validation_data=(X_test,y_test))

Epoch 1/400
46/46 [==============================] - 1s 5ms/step - loss: 24949.7070 - val_loss: 25975.6914
Epoch 2/400
46/46 [==============================] - 0s 2ms/step - loss: 24266.2852 - val_loss: 24646.0605
Epoch 3/400
46/46 [==============================] - 0s 3ms/step - loss: 21104.0801 - val_loss: 17538.1855
Epoch 4/400
46/46 [==============================] - 0s 3ms/step - loss: 11614.5508 - val_loss: 4400.1865
Epoch 5/400
46/46 [==============================] - 0s 3ms/step - loss: 7883.6294 - val_loss: 3680.8176
Epoch 6/400
46/46 [==============================] - 0s 3ms/step - loss: 7337.0708 - val_loss: 3437.5386
Epoch 7/400
46/46 [==============================] - 0s 3ms/step - loss: 7220.6694 - val_loss: 3478.0840
Epoch 8/400
46/46 [==============================] - 0s 3ms/step - loss: 7042.3320 - val_loss: 3167.4297
Epoch 9/400
46/46 [==============================] - 0s 3ms/step - loss: 6987.6396 - val_loss: 3213.4270
Epoch 10/400
46/46 [============================

In [59]:
predictions = model.predict(X_test)
sample = pd.DataFrame(predictions,columns=['Predict'])
sample['Actual']=y_test
sample.head(10)

,Predict,Actual
0,130.403305,148
1,145.266281,210
2,143.696899,136
3,122.525826,124
4,141.475525,181
5,141.348495,161
6,122.584099,166
7,128.750763,134
8,161.271378,204
9,143.014053,145


In [60]:
custom_accuracy2(sample['Actual'] , sample['Predict'],10)

21.604938271604937